In [12]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
import netCDF4
from netCDF4 import Dataset

In [13]:
# counties=gpd.read_file(r'/home/meirahwilliamson/blue/netcdf_ero/county_data/USA_Counties/USA_Counties.shp')
# counties=counties[~(counties['STATE_NAME']=='Puerto Rico') & ~(counties['STATE_NAME']=='Alaska')
#     & ~(counties['STATE_NAME']=='Hawaii')]

# #create dataframe with only FIPS
# FIPS_df = pd.DataFrame(counties["FIPS"], columns = ['FIPS']) 

In [14]:
StormData_csv=pd.read_csv(r'/blue/emullens/meirahwilliamson/StormData/StormData_corrected_no_zeros.csv')

In [15]:
D=gpd.read_file(r'/blue/emullens/meirahwilliamson/Shapefiles/US_Counties/US_Counties.shp')

In [16]:
int_damage=[]
for i in StormData_csv['DAMAGE_PROPERTY']:
    if i[-1] =='K':
        int_damage.append(float(i[:-1])*1000)
    if i[-1] =='M':
        int_damage.append(float(i[:-1])*1000000)
    if i[-1] =='B':
        int_damage.append(float(i[:-1])*1000000000)


In [17]:
crop_damage=[]
for i in StormData_csv['DAMAGE_CROPS']:
    if i[-1] =='K':
        crop_damage.append(float(i[:-1])*1000)
    if i[-1] =='M':
        crop_damage.append(float(i[:-1])*1000000)
    if i[-1] =='B':
        crop_damage.append(float(i[:-1])*1000000000)

In [18]:
StormData_csv['int_damage_crops']=crop_damage
StormData_csv['int_damage_property']=int_damage

In [19]:
idp=StormData_csv.groupby(['TextDate','TextGEOID'])['int_damage_property'].sum().reset_index()['int_damage_property']
month=StormData_csv.groupby(['TextDate','TextGEOID'])['MONTH'].first().reset_index()['MONTH']
flood_cause=StormData_csv.groupby(['TextDate','TextGEOID'])['FLOOD_CAUSE'].first().reset_index()['FLOOD_CAUSE']
date=StormData_csv.groupby(['TextDate','TextGEOID'])['FLOOD_CAUSE'].first().reset_index()['TextDate']
geoid=StormData_csv.groupby(['TextDate','TextGEOID'])['FLOOD_CAUSE'].first().reset_index()['TextGEOID']
idc=StormData_csv.groupby(['TextDate','TextGEOID'])['int_damage_crops'].sum().reset_index()['int_damage_crops']
episode_id=StormData_csv.groupby(['TextDate','TextGEOID'])['EPISODE_ID'].first().reset_index()['EPISODE_ID']
episode_narrative=StormData_csv.groupby(['TextDate','TextGEOID'])['EPISODE_NARRATIVE'].first().reset_index()['EPISODE_NARRATIVE']


In [25]:
df=pd.DataFrame({'GEOID':geoid,'date':date,'property_damage': idp,'month':month,'flood_cause':flood_cause,'episode_id':episode_id,'episode_nar':episode_narrative})


In [26]:
season=[]
for i in df['month']:
    if i == 9 or i == 10 or i == 11:
        season.append('son')
    if i == 12 or i == 1 or i == 2:
        season.append('djf')
    if i == 3 or i == 4 or i == 5:
        season.append('mam')
    if i ==6 or i == 7 or i == 8:
        season.append('jja')

In [27]:
df=pd.DataFrame({'GEOID':geoid,'date':date,'property_damage': idp,'month':month,'flood_cause':flood_cause,'season':season,'episode_id':episode_id,'episode_nar':episode_narrative})


In [28]:
df['GEOID']=df['GEOID'].astype(str).str.zfill(5)

In [29]:
df

,GEOID,date,property_damage,month,flood_cause,season,episode_id,episode_nar
0,48141,20160901,10000.0,9,Heavy Rain,son,110475,A surface high over the plains states brought ...
1,45003,20160902,1000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
2,45025,20160902,2000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
3,45027,20160902,1000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
4,45063,20160902,15000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
5,45075,20160902,10000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
6,45079,20160902,2000.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
7,45085,20160902,8500.0,9,Heavy Rain / Tropical System,son,110683,"Tropical Storm Hermine impacted the region, es..."
8,20023,20160903,40000.0,9,Heavy Rain,son,110474,During the late afternoon and through most of ...
9,20051,20160903,100000.0,9,Heavy Rain,son,110763,Thunderstorms producing heavy rainfall caused ...


In [30]:
df.to_csv(r'/blue/emullens/meirahwilliamson/StormData/StormData_corrected_no_zeros_combined.csv')

In [31]:
csv=pd.read_csv(r'/blue/emullens/meirahwilliamson/StormData/StormData_corrected_no_zeros_combined.csv')

In [32]:
csv['GEOID']=csv['GEOID'].astype(str).str.zfill(5)

In [34]:
z=D.merge(csv,how='left',on='GEOID')
z.to_file(r'/blue/emullens/meirahwilliamson/StormData_shapefile_2/StormData.shp')

In [ ]:
#loop through all files
for filename in directory:

    #open netcdf and make it a dataframe
    ERO_netcdf=xr.open_dataset(filename)
    ERO_df=ERO_netcdf.to_dataframe()

    #reset the index (bc it's currently the counties)
    ERO_df=ERO_df.reset_index()

    # use geopandas points_from_xy() to transform Longitude and Latitude into a list of shapely.Point objects and set it as a geometry while creating the GeoDataFrame
    ERO_gdf = gpd.GeoDataFrame(ERO_df, geometry=gpd.points_from_xy(ERO_df.lon, ERO_df.lat))

    # set counties coordinates to equal ERO coords
        # not degrees!! xlim, ylim in millions
            # counties = counties.to_crs(epsg=5070)
        # yes degrees!! xlim, ylim are lat, lon coords
            # counties = counties.to_crs(epsg=4326)
    ERO_gdf.crs = counties.crs

    print(ERO_gdf.crs, counties.crs)

    # spatial join ERO to counties
    counties_points_join=gpd.sjoin(counties,ERO_gdf)

    # count number of points for each risk level in each county
    group_points_county=counties_points_join.groupby(["FIPS","FCST_ERO_Surface_FULL"]).size()

    # make it a dataframe bc idxmax won't work on geodataframe, then fix the index
    group_points_county_df = pd.DataFrame(group_points_county)
    group_points_county_df=group_points_county_df.reset_index(level=['FIPS', 'FCST_ERO_Surface_FULL'])

    # find category that has the most points in each county, drop the counts, and FCST_ERO column to date
    points_county_max=group_points_county_df.loc[group_points_county_df.groupby('FIPS')['FCST_ERO_Surface_FULL'].idxmax()]
    points_county_max=points_county_max.drop(0,axis=1)
    points_county_max=points_county_max.rename(columns={"FCST_ERO_Surface_FULL": filename[19:27] })

    # merge columns onto previous columns (use merge bc not all the same length -- VA and RI)
    FIPS_df=FIPS_df.merge(points_county_max,on="FIPS",how="left")

    break;



In [52]:
#check
StormData_csv.keys()

Index(['TextDate', 'TextGEOID', 'MONTH', 'GEOID', 'BEGIN_YEARMONTH',
       'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME',
       'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME',
       'CZ_TIMEZONE', 'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY',
       'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE',
       'int_damage_property'],
      dtype='object')

In [ ]:
# apparently you also have to set the directory first -\_o_/-
os.chdir(r'/home/meirahwilliamson/blue/netcdf_ero/djf')

# choose directory
directory=os.listdir(r'/home/meirahwilliamson/blue/netcdf_ero/djf')

# run function and set variable so you can make sure it looks okay and then send it to xlsx (or something)
ero_categories_df=ero_categories(directory)

ero_categories_df.to_csv(r'/home/meirahwilliamson/blue/test/djf.csv')